### The Atmospheric and Radiometric Correction of Satellite Imagery (ARCSI) software provides a command line tool for the generation of Analysis Ready Data (ARD) optical data including atmospheric correction, cloud masking, topographic correction etc. of Earth Observation optical imagery (Blue-SWIR). The aim of ARCSI is to provide as automatic as possible method of generating Analysis Ready Data (ARD).
##### avaible at:  https://arcsi.remotesensing.info/

In [3]:
#Python Native
import csv
import os
# 3rd Party
import pandas

repository_path = '/home/user/Evaluating-the-impact-of-LaSRC-and-Sen2cor-atmospheric-correction-algorithms-on-Landsat-8-and-Sentin/'
sceneids_dir = repository_path + 'sceneids/'
dataframes_dir = repository_path + 'dataframes/'
areas = ['manaus', 'altafloresta', 'cuiaba', 'itajuba', 'riobranco']
img_input = '/user/img_folder'

for sat in ['s2', 'lc8']:
    img_in = '{}/{}/'.format(img_input, sat)
    img_out = '{}/{}_arcsi/'.format(img_input, sat)
    for area in areas:
        print("Processing {} {} ...".format(sat, area))
        with open(os.path.join(sceneids_dir, '{}_sceneids_{}.csv'.format(area, sat)), newline='') as f:
            sceneids = list(csv.reader(f))[0]
        print("Will process sceneids: {}".format(sceneids))
        my_pandas = pandas.read_csv(os.path.join(dataframes_dir, '{}_{}_pd.csv'.format(area, sat)))
        my_pandas['aot550'] = my_pandas['AOD_500nm'] * ((500/550) ** my_pandas['440-675_Angstrom_Exponent'])
        for sceneid in sceneids:
            if sat == 's2':
                satkey = 'sen2'
                headerfile = '{}.SAFE/MTD_MSIL1C.xml'.format(sceneid)
                ymd_sceneid = sceneid.split('_')[2][:8]
            else:
                satkey = 'ls8'
                headerfile = '{}/{}_MTL.txt'.format(sceneid, sceneid)
                ymd_sceneid = sceneid.split('_')[3]
            atmospro = 'Tropical'
            if area == 'itajuba' and ymd_sceneid[4:6] in ['07', '08', '09', '10']:
                atmospro = "MidlatitudeSummer"

            aot = float(my_pandas.loc[my_pandas['day'] == int(ymd_sceneid)]['aot550'])

            cmd = 'docker run --rm -v {}:/mnt/in -v {}:/mnt/out/ arcsi arcsi.py -s {} -f KEA --stats -p SREF --atmospro {} --aot {} -o /mnt/out/{}/ --scalefac 10000 -i /mnt/in/{};'.format(img_in, img_out, satkey, atmospro, aot, sceneid, headerfile)
            print(cmd)


Processing s2 manaus ...
Will process sceneids: ['S2A_MSIL1C_20180816T142041_N0206_R010_T20MRB_20180816T180323', 'S2B_MSIL1C_20180602T142039_N0206_R010_T20MRB_20180602T174131', 'S2A_MSIL1C_20171023T143321_N0206_R053_T20MRB_20171023T160302', 'S2A_MSIL1C_20161008T143312_N0204_R053_T20MRB_20161008T143313', 'S2B_MSIL1C_20180612T142039_N0206_R010_T20MRB_20180612T160557', 'S2A_MSIL1C_20160816T142322_N0204_R010_T20MRB_20160816T142325', 'S2A_MSIL1C_20160819T142752_N0204_R053_T20MRB_20160819T142755', 'S2A_MSIL1C_20171102T142751_N0206_R053_T20MRB_20171102T161043']
docker run --rm -v /dados/Rennan/arcsi/s2/:/mnt/in -v /dados/Rennan/arcsi/s2_arcsi/:/mnt/out/ arcsi arcsi.py -s sen2 -f KEA --stats -p SREF --atmospro Tropical --aot 0.11625655496637198 -o /mnt/out/S2A_MSIL1C_20180816T142041_N0206_R010_T20MRB_20180816T180323/ --scalefac 10000 -i /mnt/in/S2A_MSIL1C_20180816T142041_N0206_R010_T20MRB_20180816T180323.SAFE/MTD_MSIL1C.xml;
docker run --rm -v /dados/Rennan/arcsi/s2/:/mnt/in -v /dados/Rennan/a

ARCSI docker: https://github.com/EOFarm/docker-arcsi

A Docker image packaging Dr Pete Buntings Python Atmospheric and Radiometric Correction of Satellite Imagery (ARCSI) software (https://www.arcsi.remotesensing.info/).

This image is based on the official continuumio miniconda3 release with Python 3.5, minimal optimisation and installation of arcsi + dependencies using the conda package manager. Paths and Debian libraries required for proper functioning of ARCSI are updated.

Documentation to parameter setting at : https://py6s.readthedocs.io/en/latest/params.html


After generating your files using the arcsi docker, the output will be on .kea format

GDAL Translate (must be at least GDAL version 3) can be used to convert each output file into .tif through:

    gdal_translate -of GTIFF /data/input_img.kea /data/output_img.tif
  
or a bash "For" loop can be used to conver all .kea into .tif:

    for dir in /folder/*; do
        for file in $dir/*.kea; do
            gdal_translate -of GTIFF "$file" "${file%.*}.tif"
        done
    done